In [1]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn


from ultimate.mlp import MLP 

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)

/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(len(trainset))

4446965


In [71]:
x_train, x_val, y_train, y_val = train_test_split(trainset.drop(["winPlacePerc"],axis=1), trainset['winPlacePerc'], test_size=0.2)

# SET UP FOR MODEL ONE

## Select the trainset features for tree one 

In [79]:
x_one = x_train.drop(["matchType"],axis=1)
y_one = y_train

x_one_val = x_val.drop(["matchType"],axis=1)
y_one_val = y_val


#set the dmatrix
trainset_one_dmatrix = xgb.DMatrix(x_one.values,label=y_train.values,feature_names=x_one.columns)
valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val.values,feature_names=x_one_val.columns)

In [80]:
x_one_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
4172735,55202530414479552,23244851251017601,2281280635605269,0,0,0.0,0,0,1,54,...,766,0,4214.0,0,0.0,0,0,1109.0,2,0
1831933,18404407779974953,32594066460848237,51301979541733202,0,0,200.0,1,0,0,23,...,-1,0,0.0,0,0.0,0,0,223.8,3,1432
2415420,16056817651982391,67744720358156656,39660794403574848,0,0,0.0,0,0,0,97,...,-1,0,0.0,0,0.0,0,0,31.3,2,1541
2139248,32761683198500757,39798885019393539,30294152563504822,0,0,177.7,0,0,0,36,...,-1,0,1064.0,0,0.0,0,0,1564.0,2,1484
3413216,32655051464584690,70285655167207183,59828282538295049,1,0,194.8,1,0,2,18,...,-1,0,139.6,0,0.0,0,0,775.2,9,1481


## Configure tree one

In [81]:
params = {
    "max_depth" : 5,
    "eval_metric" : ["mae"],
}


In [82]:
iters = 20

tree_one = xgb.train(params, trainset_one_dmatrix, evals=[(trainset_one_dmatrix, "train"),(valset_one_dmatrix, 'val')], num_boost_round = iters)

[18:34:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-mae:0.201136	val-mae:0.201323
[18:34:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-mae:0.15635	val-mae:0.156381
[18:34:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-mae:0.126237	val-mae:0.126194
[18:34:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-mae:0.106902	val-mae:0.106822
[18:34:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-mae:0.09423	val-mae:0.094133
[18:34:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-mae:0.085292	val-mae:0.085165
[18:35:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 prun

xgb.plot_importance(tree_one,importance_type='cover')
xgb.plot_importance(tree_one,importance_type='weight')

# SET UP FOR MODEL TWO

## Select the trainset features for tree two

In [83]:
x_two = x_train.drop(["Id","groupId","matchId","matchType"],axis=1)
y_two = y_train

x_two_val = x_val.drop(["Id","groupId","matchId","matchType"],axis=1)
y_two_val = y_val

scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=False).fit(x_two)

In [84]:
scaler.transform(x_two)

array([[-1.        , -0.93939394, -0.92376471, ..., -0.80110769,
        -0.96610169, -1.        ],
       [-0.90909091, -0.93939394, -0.92897255, ..., -0.54584615,
        -0.95762712, -1.        ],
       [-1.        , -0.75757576, -0.89847843, ..., -0.59310769,
        -0.95762712,  0.45355191],
       ...,
       [-1.        , -0.93939394, -0.98318431, ..., -0.98263385,
        -0.98305085, -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        ,  0.4485842 ],
       [-0.90909091, -0.81818182, -0.82315294, ..., -0.68516923,
        -0.95762712, -1.        ]])

## Define the NN

In [87]:
# create NN_model
NN_model = Sequential()
NN_model.add(Dense(x_two.shape[1],  input_dim = x_two.shape[1], activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))

# output Layer
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_26 (Dense)             (None, 136)               3400      
_________________________________________________________________
dense_27 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_28 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_29 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 137       
Total params: 60,033
Trainable params: 60,033
Non-trainable params: 0
_________________________________________________________________


In [88]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [90]:
NN_model.fit(x=x_two, y=y_two, batch_size=1000,
             epochs=6, verbose=1, callbacks=callbacks_list,
             validation_split=0.20, validation_data=None, shuffle=True,
             class_weight=None, sample_weight=None, initial_epoch=0,
             steps_per_epoch=None, validation_steps=None)


Train on 2846057 samples, validate on 711515 samples
Epoch 1/6
2846057/2846057 [==============================] - 26s 9us/step - loss: 0.0680 - mean_absolute_error: 0.0680 - val_loss: 0.0685 - val_mean_absolute_error: 0.0685

Epoch 00001: val_loss improved from 0.07025 to 0.06853, saving model to Weights-001--0.06853.hdf5
Epoch 2/6
2846057/2846057 [==============================] - 24s 8us/step - loss: 0.0663 - mean_absolute_error: 0.0663 - val_loss: 0.0734 - val_mean_absolute_error: 0.0734

Epoch 00002: val_loss did not improve from 0.06853
Epoch 3/6
2846057/2846057 [==============================] - 24s 8us/step - loss: 0.0649 - mean_absolute_error: 0.0649 - val_loss: 0.0652 - val_mean_absolute_error: 0.0652

Epoch 00003: val_loss improved from 0.06853 to 0.06518, saving model to Weights-003--0.06518.hdf5
Epoch 4/6
2846057/2846057 [==============================] - 25s 9us/step - loss: 0.0641 - mean_absolute_error: 0.0641 - val_loss: 0.0649 - val_mean_absolute_error: 0.0649

Epoch 00

# ENSENBLE MODELS ON VALSET

### Predict - Update - Predict

In [91]:
def gen_predictions(models,labels,num_samples):
    labels = labels[:num_samples].values
    
    predictions = []

    for model in models:
        data = model['data']
        predictor = model['predictor']

        if model['type'] == "tree":
            matrix = xgb.DMatrix(data,feature_names=data.columns)
            prediction = predictor.predict(matrix)
        if model['type'] == "nn":
            prediction = predictor.predict(data)
        else:
            continue

        predictions.append(prediction)
        
    avg_predictions = [0.0]*len(predictions[0])
    for i in range(len(predictions)):
        for j in range(len(predictions[0])):
            avg_predictions[j] += 1.0* predictions[i][j] / len(predictions)

    print(len(avg_predictions))
    return avg_predictions
            

### Ensemble model predictions

In [92]:
models = []
models.append({'data':x_one_val,
                'predictor':tree_one,
                'type':"tree"})

models.append({'data':x_two_val,
                'predictor':NN_model,
                'type':"nn"})

labels = y_val

num_samples = 200000

predictions = gen_predictions(models,labels,num_samples)


889393


In [93]:
print(sklearn.metrics.mean_absolute_error(labels,predictions))

0.06472956851837199
